<a href="https://colab.research.google.com/github/Charee-Villapong/charee-villapong/blob/master/get_tweet_program.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
!pip install pandas
!pip install tweepy

In [11]:
# 必要なモジュールのimport
import datetime
import pandas as pd
import tweepy

In [17]:

# 各種ツイッターのキーをセット　consumer_key, consumer_secret, access_key, access_secret
consumer_key = "8ljKZRP1ekS9dzkhuaUr6mL8J"
consumer_secret = "fcpAsRKs4wg2jnUc98jJ1YG4kDLcumehO76T2CAQXH74gPeHjs"
access_key = "1289362986210009089-LYBDoTh8ipkpbCnOfy0DpaIfqQM7ZM"
access_secret = "loSkIIcmgZ7YmIx3y4IBqfTYIfN8aYc3Jva41SIFSmflP"

# 認証のためのAPIキーをセット
auth = tweepy.OAuthHandler(consumer_key,consumer_secret)
auth.set_access_token(access_key,access_secret)
api = tweepy.API(auth)
#CjtzvLBhCeBmksgJE5B1qdTzo


# API利用制限にかかった場合、解除まで待機する

'''
メインの実行部分
ツイートデータの取得からデータの出力まで
'''

# ツイートデータの取得 日付の指定は 2020-7-30のみでもOK,
# 日本時間で取得したい場合は2020-7-30_00:00:00_JSTのように指定
# JSTをつけないと時間がUTCになる UTCは協定世界時間-> JST＋9:00(日本時間よりも9時間進んでいる)

# ツイートデータを番号順に出力

# ツイートデータをDataframeにする

# ツイートデータのCSVへの出力

# ツイートを収集する関数

'''
ツイート情報を特定のキーワードで、期間を指定して収集
取得できるデータは1週間以内のデータだけ
リツイート数＋いいね数の合計でツイートを絞り込める
'''

# 検索キーワードの設定、 リツイートは除く
search_key = "Python"
# ツイートデータ取得部分
# tweepy.CursorのAPIのキーワードサーチを使用(api.search)
# qがキーワード, sinceがいつから, untilがいつまで, tweet_modeでつぶやきの省略ありなし, langで言語, .items(数)と書いてツイート数を指定
tweets = tweepy.Cursor(api.search,q=search_key,since = "2020-07-30",until = "2020-08-01",tweet_mode = "extended", lang="ja").items(100)
# ツイートのデータを取り出して、リストにまとめていく部分
print(tweets)
# ツイートデータを入れる空のリストを用意
tweet_data = []
for tweet in tweets:
    if tweet.retweet_count + tweet.favorite_count >= 50:# いいねとリツイートの合計がrlcuont以上の条件
        tweet_data.append([tweet.user_name,tweet.user.screenname,tweet.retweet_count,tweet.favorite_count,tweet.created_at.strftime("%Y-%m-%d-%H:%M-%S"),tweet.full_text.relace("\n","")])# 空のリストにユーザーネーム、スクリーンネーム、RT数、いいね数、日付などを入れる

'''
ツイートのリストを順番をつけて出力する関数の作成
'''
for tweet in tweet_data:
    print(f"({i}番目のつぶやき{tweet}")
    i += 1


'''
ツイートデータからDataFrameを作成する
'''

# データを入れる空のリストを用意(ユーザー名、ユーザーid、RT数、いいね数、日付け、ツイート本文)

# ツイートデータからユーザー名、ユーザーid、RT数、いいね数、日付け、ツイート本文のそれぞれをデータごとにまとめたリストを作る

# 先ほど作ったデータごとにまとめたリストからDataframeの作成




'\nツイートデータからDataFrameを作成する\n'